# Building an **AI Research Assistant** with the OpenAI Agents SDK

This notebook provides a reference patterns for implementing a multi‑agent AI Research Assistant that can plan, search, curate, and draft high‑quality reports with citations.

While the Deep Research feature is avaialble in ChatGPT, however, individual and companies may want to implement their own API based solution for a more finegrained control over the output.

With support for Agents, and built-in tools such as Code Interpreter, Web Search, and File Search, - Responses API makes building your own Research Assistant fast and easy. 

## Table of Contents
1. [Overview](#overview)
2. [Solution Workflow](#workflow)
3. [High‑Level Architecture](#architecture)
4. [Agent Definitions (Pseudo Code)](#agents)
    * Research Planning Agent
    * Web Search Agent
    * Knowledge Assistant Agent
    * Report Creation Agent
    * Data Analysis Agent (optional)
    * Image‑Gen Agent (optional)
5. [Guardrails & Best Practices](#best-practices)
6. [Risks & Mitigation](#risks)

### 1 — Overview <a id='overview'></a>
The AI Research Assistant helps drives better research quality and faster turnaround for knowledge content.

1. **Performs autonomous Internet research** to gather the most recent sources.
2. **Incorporates internal data sources** such as a Company's proprietery knowledge sources. 
3. **Reduces analyst effort from days to minutes** by automating search, curation and first‑draft writing.
4. **Produces draft reports with citations** and built‑in hallucination detection.

### 2 — Solution Workflow <a id='workflow'></a>
The typical workflow consists of five orchestrated steps: 

| Step | Purpose | Model |
|------|---------|-------|
| **Query Expansion** | Draft multi‑facet prompts / hypotheses | `o4-mini` |
| **Search‑Term Generation** | Expand/clean user query into rich keyword list | `gpt‑4.1` |
| **Conduct Research** | Run web & internal searches, rank & summarise results | `gpt‑4.1` + tools |
| **Draft Report** | Produce first narrative with reasoning & inline citations | `o3` |
| **Report Expansion** | Polish formatting, add charts / images / appendix | `gpt‑4.1` + tools |

### 3 — High‑Level Architecture <a id='architecture'></a>
The following diagram groups agents and tools:

* **Research Planning Agent** – interprets the user request and produces a research plan/agenda.
* **Knowledge Assistant Agent** – orchestrates parallel web & file searches via built‑in tools, curates short‑term memory.
* **Web Search Agent(s)** – perform Internet queries, deduplicate, rank and summarise pages.
* **Report Creation Agent** – consumes curated corpus and drafts the structured report.
* **(Optional) Data Analysis Agent** – executes code for numeric/CSV analyses via the Code Interpreter tool.
* **(Optional) Image‑Gen Agent** – generates illustrative figures.

Input/output guardrails wrap user prompts and final content for policy, safety and citation checks.

### 4 — Pre-requisites <a id='pre-requisites'></a>

Create a virual environment  

Install dependencies 

In [1]:
%pip install openai openai-agents --quiet

Note: you may need to restart the kernel to use updated packages.


### 5 — Agents (Pseudo Code) <a id='agents'></a>
Below are skeletal class definitions illustrating how each agent’s policy and tool‑usage might look.

#### Step 1 - Query Expansion

The query expansion step ensures the subsequent agents conducting research have sufficient context of user's inquiry. 

The first step is to understand user's intent, and make sure the user has provided sufficinet details for subsequent agents to search the web, build a knowledge repository, and prepare a deepdive report. The `query_expansion_agent.py` accomplishes this with the prompt that outlines minimum information needed from the user to generate a report. This could include timeframe, industry, target audience, etc. The prompt can be tailored to the need of your deepresearch assistant. The agent will put a `is_task_clear` yes or no, when its no, it would prompt the user with additional questions, if sufficent information is available, it would output the expanded prompt. 

This is also an opportunity to enforce input guardrails for any research topics that you'd like to restrict the user from reserarching based on your usage policies. 

##### Input Guardrails with Agents SDK 
Let's assume our ficticious guardrail is to prevent the user from generating a non-AI releated topic report. For this we will define a guardrail agent. The guardrail agent `topic_guradrail.py` checks whether the topic is related to AI, if not, it raises an execption. The function `ai_topic_guardrail` is passed to the `QueryExpansionAgent()` as `input_guardrails`

In [2]:
from ai_research_assistant_resources.agents_tools_registry.query_expansion_agent import QueryExpansionAgent
from agents import InputGuardrailTripwireTriggered

query_expansion_agent_guardrail_check = QueryExpansionAgent()

try:

    result = await query_expansion_agent_guardrail_check.task("Write a research report on the latest trends in luxury goods market")

except InputGuardrailTripwireTriggered as e:
    reason = e.guardrail_result.output.output_info.reasoning
    #            └─────┬─────┘
    #            GuardrailFunctionOutput
    print("🚫 Guardrail tripped – not an AI topic:", reason)


🚫 Guardrail tripped – not an AI topic: The user's request focuses on the luxury goods market, which pertains to market trends in the luxury sector rather than artificial intelligence. Therefore, it is not about AI.


In [3]:
from ai_research_assistant_resources.agents_tools_registry.query_expansion_agent import QueryExpansionAgent

query_expansion_agent = QueryExpansionAgent()

# Initial prompt to the agent
prompt: str = "Draft a research report on the latest trends in AI developments"
expanded_query = "" 

try: 

    while True:
        # Execute the agent with the current prompt
        result = await query_expansion_agent.task(prompt)

        # When the task is clear, show the expanded query and exit.
        if result.is_task_clear == "yes":
            expanded_query = result.expanded_query
            print("\nExpanded query:\n", expanded_query)
            break

        # Otherwise, display the clarifying questions and ask the user for input.
        print("\nThe task is not clear. The agent asks:\n", result.questions)
        prompt = input("Please provide the missing details so I can refine the query: ")
        print("\n")
        print("user input: ", prompt)
        

except Exception as e:
    print("Non-AI topic guardrail tripped!", e)


The task is not clear. The agent asks:
 1. What timeframe should the report cover (e.g., the past year, the past five years, up to current date)?
2. Should the report focus on specific AI subfields (e.g., natural language processing, computer vision, reinforcement learning) or provide a general overview?
3. Are there particular industries or application domains (e.g., healthcare, finance, manufacturing) you want the report to emphasize?
4. What length or depth do you expect for the report (e.g., a brief summary, a detailed 20-page analysis)?
5. Who is the target audience for the report (e.g., technical researchers, business executives, policymakers)?


user input:  5 years, AI in healthcare, exec summary 

Expanded query:
 Draft an executive summary research report on the latest trends in AI developments in healthcare over the past five years. Summarize key advancements across major subfields such as diagnostic imaging, predictive analytics, natural language processing for clinical do

#### Step 2 - Web Search Terms 

Conducting Web search is typically an integral part of the deep research process. First we generate web search terms relevant to the research report. In the next step we will search the web and build a knowledge repository of the data.

The `WebSearchTermsGenerationAgent` takes as input the the expanded prompt, and generates succient search terms. You can structure the search term generation prompt according to your user's typical requirements such as include adjacent industries in the search terms, include competitors, etc. Additionally, you can also control how much data you want to gather e.g., number of search terms to generate. In our case, we will limit to 3 search terms. 

In [4]:
from ai_research_assistant_resources.agents_tools_registry.web_search_terms_generation_agent import WebSearchTermsGenerationAgent

search_terms_agent = WebSearchTermsGenerationAgent(3)

result = await search_terms_agent.task(expanded_query)

search_terms_raw = result

for i, query in enumerate(search_terms_raw.Search_Queries, start=1):
    print(f"{i}. {query}")

1. Latest AI trends in healthcare 2025 report
2. Advancements in AI for diagnostic imaging and predictive analytics 2020-2025
3. Impactful AI case studies in healthcare and market adoption analysis 2025


#### Step 3 - Scroll the Web build a inventory of data sources 

We will use custom web search to identify and knowledge content to form the baseline for our report. You can learn more about building custom web search and retreival here. [Building a Bring Your Own Browser (BYOB) Tool for Web Browsing and Summarization](https://cookbook.openai.com/examples/third_party/web_search_with_google_api_bring_your_own_browser_tool). You will also need a Google Custom Search API key and Custom Search Engine ID (CSE ID) in a .env file at the root. 

NOTE: The reason for using custom web search is provide more finegrained control over which information is retreived, and guardrails such as excluding competitor's content from your report. 

This is a 3 step process: 

1. Obtain the search results (top 10 pages)
2. Scroll the pages, and summarize the key points 
3. Output guardrails to weedout irrelevant or undesirable results (e.g., the timeframe of the content doesn't align with user's need, or mentions a competitor)

prerequisite pip install nest_asyncio

In [5]:
from ai_research_assistant_resources.utils.web_search_and_util import get_results_for_search_term
import json

research_results = []

for idx, query in enumerate(search_terms_raw.Search_Queries, 1):
    print(f"Search Query {idx}: {query}")
    research_results.append(get_results_for_search_term(query))

if research_results:                       
    with open("research_results.json", "w", encoding="utf-8") as f:
        json.dump(research_results, f, indent=2, ensure_ascii=False)
    print("Results written to research_results.json")
else:
    print("No results returned.")


Search Query 1: Latest AI trends in healthcare 2025 report
Search Query 2: Advancements in AI for diagnostic imaging and predictive analytics 2020-2025
Search Query 3: Impactful AI case studies in healthcare and market adoption analysis 2025
Results written to research_results.json


### Step-4: Create a report

In [6]:
from ai_research_assistant_resources.agents_tools_registry.report_writing_agent import (
    ReportWritingAgent,
)
import json
from pathlib import Path

# ------------------------------------------------------------------
# 1. Load research results
# ------------------------------------------------------------------
with open("research_results.json", "r", encoding="utf-8") as f:
    research_results = f.read()

# ------------------------------------------------------------------
# 2. Draft the report
# ------------------------------------------------------------------
outline = """ Draft a comprehensive research report analyzing the latest trends in artificial intelligence (AI) developments within the healthcare industry over the past five years. The report should evaluate advancements in machine learning, deep learning, natural language processing, medical imaging, and other relevant AI applications, while also examining regulatory, ethical, and operational impacts on healthcare delivery. Include detailed case studies, emerging research areas, and recommendations for future innovation in the industry."""  # ← customise as needed

report_agent = ReportWritingAgent(research_resources=research_results)

draft_md = await report_agent.task(outline)

# ------------------------------------------------------------------
# 4. Persist to file
# ------------------------------------------------------------------
Path("REPORT_DRAFT.md").write_text(draft_md, encoding="utf-8")
print("✅ Report written to REPORT_DRAFT.md")

✅ Report written to REPORT_DRAFT.md


### Step-5: Report Expansion or Scouting for additional data points (OPTIONAL)

If you have a large corpus of data, you may have a secondary report expansion agent review each section of the report, and add content that may have been overlooked in the first pass by the report writer. This can be selectively done for a section, or for all sections based on your use case. 

While it is beyond the purview of this Cookbook, the overall architecture is as follows. 

### Step-6: Organize the with References and Table of Content 

We let the LLM focus on generating the content, the content formatting such as creating a Table of Content upfront, and move references to the end.  



In [7]:
import re

def update_references(file_path, search_results_json):
    """
    Update the references in a Markdown file by extracting unique URLs from <source></source> tags and creating a
    References section at the end of the file.

    :param file_path: The path to the Markdown file.
    """
    global content
    # Read the markdown_content of the MD file

    global url_to_title
    # Load the search results
    # Create a dictionary for quick lookup of titles by URL
    url_to_title = {entry["URL"]: entry["title"] for entry in search_results_json}

    with open(file_path, 'r') as file:
        content = file.read()
    # Remove the existing References section if it exists
    content = re.sub(r'\n## References[\s\S]*', '', content)
    content = re.sub(r'\n### References[\s\S]*', '', content)

    # Find all <source></source> tags and extract the URLs
    sources = re.findall(r'<source>(.*?)</source>', content)
    # Eliminate duplicates while maintaining order
    unique_sources = []
    unique_references = {}
    for source in sources:
        if source not in unique_sources:
            unique_sources.append(source)
        unique_references[source] = unique_sources.index(source) + 1
    # Create the References section
    references_section = "\n\n## References\n"

    for i, source in enumerate(unique_sources, start=1):
        title = url_to_title.get(source, "Source not found in the search results")
        references_section += f"{i}. [{title}]({source})\n"
        # references_section += f"{i}. {source}\n"

    # Replace <source></source> tags with [reference #]
    for source, reference_number in unique_references.items():
        # markdown_content = markdown_content.replace(f'<source>{source}</source>', f'[reference {reference_number}]')
        content = content.replace(f'<source>{source}</source>', f'<sup>[[{reference_number}]({source})]</sup>')
    # Append the References section to the markdown_content
    content += references_section
    # Save the modified markdown_content back to the file
    with open(file_path, 'w') as file:
        file.write(content)
        
        
def add_toc_to_markdown(file_path):
    """
    Add a Table of Contents (TOC) to a Markdown file by generating links to the headings in the file.

    :param file_path: The path to the Markdown file.
    """

    def generate_toc_line(line):
        level = line.count('#') - 2
        heading = line.strip().lstrip('#').strip()
        link = heading.lower().replace(' ', '-').replace('.', '').replace(',', '')
        return f"{'  ' * level}- [{heading}](#{link})\n"

    with open(file_path, 'r') as file:
        lines = file.readlines()

    toc_lines = []
    content_start_index = 0
    for i, line in enumerate(lines):
        if line.startswith('## '):
            content_start_index = i
            break

    for line in lines[content_start_index:]:
        if line.startswith('## ') or line.startswith('### '):
            toc_lines.append(generate_toc_line(line))

    toc_content = "# Table of Contents\n" + ''.join(toc_lines) + "\n---\n\n"
    new_content = toc_content + ''.join(lines)

    with open(file_path, 'w') as file:
        file.write(new_content)

### 5 — Guardrails & Best Practices <a id='best-practices'></a>
* **Crawl → Walk → Run**: start with a single agent, then expand into a swarm. 
* **Expose intermediate reasoning** (“show the math”) to build user trust.  
* **Parameterise UX** so analysts can tweak report format and source mix. 
* **Native OpenAI tools first** (web browsing, file ingestion) before reinventing low‑level retrieval. 

### 6 — Risks & Mitigation <a id='risks'></a>
| Pitfall | Mitigation |
|---------|------------|
| Scope‑creep & endless roadmap | Narrow MVP & SMART milestones | fileciteturn1file4L23-L24 |
| Hallucinations & weak guardrails | Golden‑set evals, RAG with citation checks | fileciteturn1file4L25-L26 |
| Run‑away infra costs | Cost curve modelling; efficient models + autoscaling | fileciteturn1file4L27-L28 |
| Talent gaps | Upskill & leverage Agents SDK to offload core reasoning | fileciteturn1file4L29-L30 |